In [54]:
import sys;
import py_ecc.bn128
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, neg, pairing, Z1, Z2
import galois
from functools import reduce
import numpy as np
from operator import add


def evaluate_polynomial_galois(coefs, x, GF):
    result = GF(0)
    for coef in coefs:
        result = result * x + GF(coef)
    return int(result)

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

In [72]:
def evaluate_poly(poly, trusted_points, verbose=False):
    coeff = poly.coefficients()[::-1]

    #assert len(coeff) == len(trusted_points), "Polynomial degree mismatch!"


    terms = [multiply(point, int(coeff))
             for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    return evaluation

In [4]:
GF = galois.GF(curve_order)

In [61]:
GF=galois.GF(101)

In [122]:
## TRUSTED SETUP
def trusted_setup(U,V,W,l):
 
    m=len(U)-1
    n=len(U[0])

    print("m (#var) = ",m)
    print("n (#eq)  = ",n)

    #=================================Pour trouver t(x)  ==========================
    values = [i for i in range(1, len(U[0])+1)]
    poly_t = galois.Poly([1], field=GF)
    for val in values:
        poly_t *= galois.Poly([1, -val], field=GF)
    t=[]
    for elem in poly_t.coeffs:
        t.append(int(elem))
    print("Polynome t= ",t, " donc t(x) = ",poly_t)
    #==============================================================================
    
    x=GF(75)
    print("x = ",int(x))

    t_x=evaluate_polynomial_galois(t, x, GF)
    print("t(x) = ",t_x)

    alpha = GF(2)
    beta  = GF(3)
    gamma = GF(5)
    delta = GF(11)

    print("alpha = ",int(alpha))
    print("beta = ",int(beta))
    print("gamma = ",int(gamma))
    print("delta = ",int(delta))

    x_power_i=[]
    for i in range(0, n):
        x_power_i.append(x**i)

    U_polys=[]
    V_polys=[]
    W_polys=[]

    for elem in U:
        U_polys.append(galois.Poly((elem),field=GF))
    for elem in V:
        V_polys.append(galois.Poly((elem),field=GF))
    for elem in W:
        W_polys.append(galois.Poly((elem),field=GF))

    private_polys=[]
    for i in range(0, l+1): #l+1 car on inclut l on veut [0,l]
        u_i=U_polys[i](x)
        v_i=V_polys[i](x)
        w_i=W_polys[i](x)
        private_polys.append((int(beta)*u_i+int(alpha)*v_i+w_i)/gamma)
    public_polys=[]
    for i in range(l+1, m+1): # on veut [l+1,m-1]
        u_i=U_polys[i](x)
        v_i=V_polys[i](x)
        w_i=W_polys[i](x)
        public_polys.append((int(beta)*u_i+int(alpha)*v_i+w_i)/delta)
    x_power_i_t_x=[]
    for i in range(0,n-2+1):#[0,n-1]
        x_power_i_t_x.append((x**i)*t_x/delta)
    G1_alpha = multiply(G1, int(alpha))
    G1_beta = multiply(G1, int(beta))
    G1_delta = multiply(G1, int(delta))
    G1_x_power_i = [multiply(G1,int(elem)) for elem in x_power_i]
    G1_private_polys = [multiply(G1,int(elem)) for elem in private_polys]
    G1_public_polys = [multiply(G1,int(elem)) for elem in public_polys]
    G1_x_power_i_t_x = [multiply(G1,int(elem)) for elem in x_power_i_t_x]
    
    output_1=[G1_alpha, G1_beta, G1_delta, G1_x_power_i,G1_x_power_i_t_x,G1_public_polys,G1_private_polys]

    G2_beta = multiply(G2, int(beta))
    G2_gamma = multiply(G2, int(gamma))
    G2_delta = multiply(G2, int(delta))
    G2_x_power_i = [multiply(G2,int(elem)) for elem in x_power_i]

    output_2=[G2_beta, G2_gamma, G2_delta, G2_x_power_i]
    return output_1, output_2

In [148]:
## PROVER
def prover(U,V,W,l,sigma_1,sigma_2,a):
    print("\n============= Prover =============")
    curve_order=101

    r=GF(13)
    s=GF(17)

    m=len(U)-1
    n=len(U[0])

    G1_alpha, G1_beta, G1_delta, G1_x_power_i,G1_x_power_i_t_x,G1_public_polys,G1_private_polys = sigma_1
    G2_beta, G2_gamma, G2_delta, G2_x_power_i = sigma_2

    #=================================Pour trouver t(x)  ==========================
    values = [i for i in range(1, len(U[0])+1)] # Car on a 6 colonnes
    poly_t = galois.Poly([1], field=GF)  # Start with the polynomial '1' in the given field
    for val in values:
        poly_t *= galois.Poly([1, -val], field=GF)
    t=[]
    for elem in poly_t.coeffs:
        t.append(int(elem))
    #=================================Pour trouver h(x)  ==========================
    U_polys=[]
    V_polys=[]
    W_polys=[]

    for elem in U:
        U_polys.append(galois.Poly((elem),field=GF))
    for elem in V:
        V_polys.append(galois.Poly((elem),field=GF))
    for elem in W:
        W_polys.append(galois.Poly((elem),field=GF))

    witness = a
    Ua = inner_product_polynomials_with_witness(U_polys, witness)
    Va = inner_product_polynomials_with_witness(V_polys, witness)
    Wa = inner_product_polynomials_with_witness(W_polys, witness)

    t_poly=galois.Poly(t,field=GF)
    print("t_poly(x) = ",t_poly)

    h = (Ua * Va - Wa) // t_poly
    h_rem = (Ua * Va - Wa) % t_poly

    assert h_rem == 0, "h(x) is not a multiple of t(x)"

    h_t_x=[]
    for i in range(0, len(h.coeffs)):
        h_t_x.append(multiply(G1_x_power_i_t_x[i],int(h.coeffs[::-1][i])))
    print("h_t_x=",h_t_x)

    #==============================================================================

    Ui_x = []
    for i in range(0, m+1):
        temp=None
        for j in range(0, len(U[i])):
            temp2=multiply(G1_x_power_i[j],int(U[i][::-1][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        Ui_x.append(temp)

    A_1=None
    for i in range(0, m+1):
        temp=multiply(Ui_x[i],int(a[i]))
        A_1=py_ecc.bn128.add(A_1,temp)
    A_1=py_ecc.bn128.add(A_1,G1_alpha)
    r_delta=multiply(G1_delta,int(r))
    A_1=py_ecc.bn128.add(A_1,r_delta)
    print("A_1=",A_1)

    V_i_x_1 = []
    for i in range(0, m+1):
        temp=None
        for j in range(0, len(V[i])):
            temp2=multiply(G1_x_power_i[j],int(V[i][::-1][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        V_i_x_1.append(temp)

    V_i_x_2 = []
    for i in range(0, m+1):
        temp=None
        for j in range(0, len(V[i])):
            temp2=multiply(G2_x_power_i[j],int(V[i][::-1][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        V_i_x_2.append(temp)

    B_1=None
    for i in range(0, m+1):
        temp=multiply(V_i_x_1[i],int(a[i]))
        B=py_ecc.bn128.add(B_1,temp)
    B_1=py_ecc.bn128.add(B_1,G1_beta)
    s_gamma=multiply(G1_delta,int(s))
    B_1=py_ecc.bn128.add(B_1,s_gamma)
    print("B_1=",B_1)

    B_2=None
    for i in range(0, +1):
        temp=multiply(V_i_x_2[i],int(a[i]))
        B=py_ecc.bn128.add(B_2,temp)
    B_2=py_ecc.bn128.add(B_2,G2_beta)
    s_gamma=multiply(G2_gamma,int(s))
    B_2=py_ecc.bn128.add(B_2,s_gamma)
    print("B_2=",B_2)

    # A = alpha_G1 + multiply(delta_G1,r)
    # for i in range(0, m):
    #     A+= a[i] * (U_polys[i](tau))
    #     A= add(A, multiply(x_power_i_G1[i],s))

    # C_1=GF(0)
    # for i in range(2, m):
    #     C_1+=a[i]*(beta * (U_polys[i](tau)) + alpha * (V_polys[i](tau)) + W_polys[i](tau))
    # C_1=C_1 / delta
    # C_2=(h_quo(tau) * t_poly(tau)) / delta
    # C_3=(A * s + B * r - r * s * delta)
    # C=C_1+C_2+C_3
    # print("C_bis=",C)

    # A_1=multiply(G1,int(A)) 
    # C_1=multiply(G1,int(C))
    # B_2=multiply(G2,int(B))

    # print("A_1=",A_1)
    # print("C_1=",C_1)
    # print("B_2=",B_2)

    #Le prover va donner A_1, C_1 et B_2 

In [149]:
## VERIFIER

def verifier(A_1, C_1, B_2):
    D=GF(0)
    for i in range(0, 2):
        D+=a[i]*(beta * (U_polys[i](tau)) + alpha * (V_polys[i](tau)) + W_polys[i](tau))
    D=multiply(G1,int(D/gamma))
    print("D_1= ",D)

    lhs = pairing(B_2, A_1) #A_1*B_2
    print("lhs=",lhs)

    rhs_1 = pairing(multiply(G2,int(beta)),multiply(G1,int(alpha)))
    rhs_2 = pairing(multiply(G2,int(gamma)),D)
    rhs_3 = pairing(multiply(G2,int(delta)),C_1)
    rhs=rhs_1*rhs_2*rhs_3
    print("rhs=",rhs)
    print(lhs == rhs)


In [150]:
## DATA 
import numpy as np

def main():
   # x^3+x+5
   # Ensemble des varibles = ['1', 'out', 'x', 'v1', 'v2', 'v3']
   # U=[
   #     [3648040478639879203707734290876212514758060733402672390616367364429301415937, 21888242871839275222246405745257275088548364400416034343698204186575808495612, 18240202393199396018538671454381062573790303667013361953081836822146507079690, 21888242871839275222246405745257275088548364400416034343698204186575808495612], 
   #     [0, 0, 0, 0], 
   #     [14592161914559516814830937163504850059032242933610689562465469457717205663744, 5, 7296080957279758407415468581752425029516121466805344781232734728858602831861, 8], 
   #     [10944121435919637611123202872628637544274182200208017171849102093287904247809, 21888242871839275222246405745257275088548364400416034343698204186575808495613, 10944121435919637611123202872628637544274182200208017171849102093287904247818, 21888242871839275222246405745257275088548364400416034343698204186575808495611], 
   #     [10944121435919637611123202872628637544274182200208017171849102093287904247808, 10944121435919637611123202872628637544274182200208017171849102093287904247812, 21888242871839275222246405745257275088548364400416034343698204186575808495610, 4], 
   #     [18240202393199396018538671454381062573790303667013361953081836822146507079681, 21888242871839275222246405745257275088548364400416034343698204186575808495616, 3648040478639879203707734290876212514758060733402672390616367364429301415938, 21888242871839275222246405745257275088548364400416034343698204186575808495616]]

   # V=[
   #     [7296080957279758407415468581752425029516121466805344781232734728858602831872, 10944121435919637611123202872628637544274182200208017171849102093287904247811, 3648040478639879203707734290876212514758060733402672390616367364429301415931, 3], 
   #     [0, 0, 0, 0], 
   #     [14592161914559516814830937163504850059032242933610689562465469457717205663745, 10944121435919637611123202872628637544274182200208017171849102093287904247806, 18240202393199396018538671454381062573790303667013361953081836822146507079686, 21888242871839275222246405745257275088548364400416034343698204186575808495615], 
   #     [0, 0, 0, 0], 
   #     [0, 0, 0, 0], 
   #     [0, 0, 0, 0]]

   # W=[
   #     [0, 0, 0, 0], 
   #     [18240202393199396018538671454381062573790303667013361953081836822146507079681, 21888242871839275222246405745257275088548364400416034343698204186575808495616, 3648040478639879203707734290876212514758060733402672390616367364429301415938, 21888242871839275222246405745257275088548364400416034343698204186575808495616], 
   #     [0, 0, 0, 0], 
   #     [3648040478639879203707734290876212514758060733402672390616367364429301415936, 10944121435919637611123202872628637544274182200208017171849102093287904247810, 7296080957279758407415468581752425029516121466805344781232734728858602831868, 4], 
   #     [10944121435919637611123202872628637544274182200208017171849102093287904247809, 21888242871839275222246405745257275088548364400416034343698204186575808495613, 10944121435919637611123202872628637544274182200208017171849102093287904247818, 21888242871839275222246405745257275088548364400416034343698204186575808495611], 
   #     [10944121435919637611123202872628637544274182200208017171849102093287904247808, 10944121435919637611123202872628637544274182200208017171849102093287904247812, 21888242871839275222246405745257275088548364400416034343698204186575808495610, 4]
   # ]

   U=[[ 73,   0,  64  , 0  , 0,  16,  16,  16],
[ 50 ,  0 , 12 ,  0 ,  0 , 63,  63,  63],
[ 40   ,0  ,97  , 0  , 0 , 47 , 47  ,47],
[ 72   ,0 , 74,   0 ,  0 , 36 , 36 , 36],
[ 26  , 0  ,66  , 0  , 0 , 41 , 41 , 41],
[ 42   ,0 , 92 ,  0 ,  0 ,100 ,100 ,100]]
   
   V=[[ 16 ,  0,   5 , 38,  42,   0,   0,  0],
[ 63 ,  0,  85 ,  2,  52,   0,   0,   0],
[ 47 ,  0,  20 ,  2,  32,   0,   0,   0],
[ 36  , 0 , 83  ,52 , 31 ,  0 ,  0 ,  0],
[ 41  , 0 , 99  ,37 , 25 ,  0 ,  0 ,  0],
[100  , 0 , 12  ,71 , 20 ,  0 ,  0 ,  0]]
   
   W=[[  0  ,16   ,0  ,85  ,80  ,42  ,59  ,21],
[  0  ,63   ,0  ,17  ,96  ,52  , 7  ,68],
[  0  ,47   ,0  ,45  ,52  ,32  ,51  ,76],
[  0  ,36   ,0  ,89  ,86  ,31  ,67  ,95],
[  0 , 41  , 0 , 62 ,  4 , 25 , 33 , 37],
[  0 ,100   ,0  , 6  ,86  ,20  ,86  , 6]]

   W=GF(np.transpose(W))
   U=GF(np.transpose(U))
   V=GF(np.transpose(V))

   print("U=",U,"\n")
   print("V=",V,"\n")
   print("W=",W,"\n")
   
   p=101

   # a = [GF(1%p), GF(553%p), GF(5%p), GF(25%p), GF(125%p), GF(375%p), GF(125%p), GF(50%p)]

   a=[1 ,48 , 5 ,25, 24, 72, 24, 50]
   a=GF(a)

   l=1 # car dans a=[1, 135, 5, 25, 125, 130] on donne en public les 2 premiers éléments de a et le reste c'est privée
   sigma_1,sigma_2= trusted_setup(U,V,W,l)

   G1_alpha, G1_beta, G1_delta, G1_x_power_i, G1_x_power_i_t_x, G1_public_polys, G1_private_polys = sigma_1
   G2_beta, G2_gamma, G2_delta, G2_x_power_i = sigma_2
   
   print("dim :",len(G1_alpha)," | G1_alpha=",G1_alpha)
   print("dim :",len(G1_beta)," | G1_beta=",G1_beta)
   print("dim :",len(G1_delta)," | G1_delta=",G1_delta)
   print("dim :",len(G1_x_power_i)," | G1_x_power_i=",G1_x_power_i)
   print("dim :",len(G1_x_power_i_t_x)," | G1_x_power_i_t_x=",G1_x_power_i_t_x)
   print("dim :",len(G1_public_polys)," | G1_public_polys=",G1_public_polys)
   print("dim :",len(G1_private_polys)," | G1_private_polys=",G1_private_polys)

   print("dim :",len(G2_beta)," | G2_beta=",G2_beta)
   print("dim :",len(G2_gamma)," | G2_gamma=",G2_gamma)
   print("dim :",len(G2_delta)," | G2_delta=",G2_delta)
   print("dim :",len(G2_x_power_i)," | G2_x_power_i=",G2_x_power_i)
   # print("Sigma_1")
   # for elem in sigma_1:
   #     print(elem)
   # print(" ")
   # print("Sigma_2")
   # for elem in sigma_2:
   #     print(elem)

   prover(U,V,W,l,sigma_1,sigma_2,a)


main()


U= [[ 73  50  40  72  26  42]
 [  0   0   0   0   0   0]
 [ 64  12  97  74  66  92]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [ 16  63  47  36  41 100]
 [ 16  63  47  36  41 100]
 [ 16  63  47  36  41 100]] 

V= [[ 16  63  47  36  41 100]
 [  0   0   0   0   0   0]
 [  5  85  20  83  99  12]
 [ 38   2   2  52  37  71]
 [ 42  52  32  31  25  20]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]] 

W= [[  0   0   0   0   0   0]
 [ 16  63  47  36  41 100]
 [  0   0   0   0   0   0]
 [ 85  17  45  89  62   6]
 [ 80  96  52  86   4  86]
 [ 42  52  32  31  25  20]
 [ 59   7  51  67  33  86]
 [ 21  68  76  95  37   6]] 

m (#var) =  7
n (#eq)  =  6
Polynome t=  [1, 80, 74, 73, 8, 54, 13]  donc t(x) =  x^6 + 80x^5 + 74x^4 + 73x^3 + 8x^2 + 54x + 13
x =  75
t(x) =  58
alpha =  2
beta =  3
gamma =  5
delta =  11
dim : 2  | G1_alpha= (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515

In [151]:

#Le prouver gen r,s deux randoms pour avoir du ZKP
r=GF(30)
s=GF(200)

h_tau=evaluate_polynomial_galois(h_quo.coeffs, tau, GF)
ht_tau=h_tau*t_tau
print("ht(tau)=",ht_tau)

#===============================CALCUL DE A============================================
A_G1=None
A_0=alpha_G1

A_1=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_U_tau_G1=multiply(G1, eval_U_tau)
    eval_aU_tau_G1=multiply(eval_U_tau_G1, int(a[i]))
    A_1=add(A_1, eval_aU_tau_G1)

A_2=multiply(delta_G1, int(r))

A_G1=add(A_0, add(A_1, A_2))
print("A_G1=",A_G1)

#===============================CALCUL DE B_2============================================
B_G2=None
B_0=beta_G2

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G2=multiply(G2, eval_V_tau)
    eval_aV_tau_G2=multiply(eval_V_tau_G2, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G2)

B_2=multiply(delta_G2, int(s))

B_G2=add(B_0, add(B_1, B_2))
print("B_G2=",B_G2)

#===============================CALCUL DE B_1============================================
B_G1=None
B_0=beta_G1

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G1=multiply(G1, eval_V_tau)
    eval_aV_tau_G1=multiply(eval_V_tau_G1, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G1)

B_2=multiply(delta_G1, int(s))

B_G1=add(B_0, add(B_1, B_2))
print("B_G1=",B_G1)

#===============================CALCUL DE C ============================================
C=None

C_0=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_W_tau=evaluate_polynomial_galois(W[i], tau, GF)
    temp=int(beta)*int(eval_U_tau)+int(alpha)*int(eval_V_tau)+int(eval_W_tau)
    eval_G1=multiply(G1, temp)
    eval_aW_tau_G1=multiply(eval_G1, int(a[i]))
    C_0=add(C_0, eval_aW_tau_G1)
C_1=None
C_2=None
C_3=None
C_4=None

ValueError: GF(101) scalars must be in `0 <= x < 101`, not 200.

In [ ]:

#Le prouver gen r,s deux randoms pour avoir du ZKP
r=GF(30)
s=GF(200)

h_tau=evaluate_polynomial_galois(h_quo.coeffs, tau, GF)
ht_tau=h_tau*t_tau
print("ht(tau)=",ht_tau)

#===============================CALCUL DE A============================================
A_G1=None
A_0=int(alpha)

A_1=0
for i in range(0, nb_eq):
    A_1+=int(a[i])*evaluate_polynomial_galois(U[i], tau, GF)

A_2=int(r)*int(delta)

A_G1=A_0+A_1+A_2
print("A_G1=",A_G1)
print("A_G1=",multiply(G1,A_G1))


#===============================CALCUL DE B_2============================================
B_G2=None
B_0=beta_G2

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G2=multiply(G2, eval_V_tau)
    eval_aV_tau_G2=multiply(eval_V_tau_G2, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G2)

B_2=multiply(delta_G2, int(s))

B_G2=add(B_0, add(B_1, B_2))
print("B_G2=",B_G2)

#===============================CALCUL DE B_1============================================
B_G1=None
B_0=beta_G1

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G1=multiply(G1, eval_V_tau)
    eval_aV_tau_G1=multiply(eval_V_tau_G1, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G1)

B_2=multiply(delta_G1, int(s))

B_G1=add(B_0, add(B_1, B_2))
print("B_G1=",B_G1)

#===============================CALCUL DE C ============================================
C=None

C_0=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_W_tau=evaluate_polynomial_galois(W[i], tau, GF)
    temp=int(beta)*int(eval_U_tau)+int(alpha)*int(eval_V_tau)+int(eval_W_tau)
    eval_G1=multiply(G1, temp)
    eval_aW_tau_G1=multiply(eval_G1, int(a[i]))
    C_0=add(C_0, eval_aW_tau_G1)
C_1=None
C_2=None
C_3=None
C_4=None